Demographic Issues in PLAICRAFT AI - Equity as a Cornerstone to Building Better AI-powered NPCS
====
![My Image](https://www.pluggedin.com/wp-content/uploads/2020/01/minecraft-review-image-1536x881.jpg)

Introduction
---

In September of 2023, AI Researchers at UBC launched a “free Minecraft in the cloud” AI research data collection project. The project aims to collect 10,000 hours of freely volunteered play time. **Using this data, the lab hopes to create artificially intelligent non-player characters in Minecraft that act like real people. These NPCs would respond to both visual and auditory stimulus, and be able to play in various game modes just like real people**.

https://plai.cs.ubc.ca/2023/09/27/plaicraft/

 
Minecraft, a video game released in 2009, has taken the world by storm due to its limitless "sandbox game" design. There is no one right way to play. Minecraft can be played alone or in groups, with friends or online, passively or violently. Some gamemodes involve survival of the fittest, while others, such as "Creative" mode, use the game to build whatever the user wants (https://www.minecraft.net/en-us/article/what-minecraft).

**Research has shown different demographics approach playing video games differently.** Due to literacy and generational conventions, different age-groups use in-game chat features differently (https://morganya.org/research/ames-cscw17-minecraft.pdf); In studies examining gender differences, girls often played more collaboratively and patiently, while boys were drawn to objective-oriented gameplay modes (https://www.jstor.org/stable/26511549). 


**When specifically considering Minecraft's diverse gamemodes, it makes sense these demographic differences would be refected in users' choices and approaches to limitless options**. 

If PLAICRAFT AI aims to make *realistic* non-player characters, the demographic of its research should be equitable and diverse– and by extension, include all types and approaches to gameplay. We were provided with a dataset which contained seven observations about each of its 196 unique players. Using this, our investigation seeks to better understand the demographic of participants in the research project so far, and ultimately to highlight biases and deficits that might skew the behaviour of PLAICRAFT's AI-Powered NPCs.


More specifically, we'd like to know: 

> Can factors such as self-described experience and age successfully predict the gender of players in this data set?

If these factors are accurate predictors, there would be evidence to suggest that different gender demographics engage with Minecraft differently. If there's evidence of a difference in demographic behaviour, effort should be put towards targeting them equitably for a more realistic AI result. 



Methods & Results:
==
describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
your report should include code which:
- loads data 
- wrangles and cleans the data to the format necessary for the planned analysis
- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
- performs the data analysis
- creates a visualization of the analysis 
note: all figures should have a figure number and a legend


Discussion:
==
- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?

References:
==
You may include references if necessary, as long as they all have a consistent citation style.

In [ ]:
#PLAICRAFT Data Analysis - following my proposal, doing a KNN regression using every trait in the players data to see if we can correctly classify gender

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
library(RColorBrewer)
options(repr.matrix.max.rows = 6)

In [ ]:

#experience is now scored from 1 to 4, with 1 being the most beginner and 4 being the most advanced
#subscription is now scored from 0 to 1, where 0 is not subscribed and 1 is subscribed
players <- read_csv("https://raw.githubusercontent.com/ellailan/PLAICRAFT-Data-Analysis/refs/heads/main/data/players.csv") |>
            filter(gender == "Male" | gender == "Female") |>  #for the sake of simplicity.... but could be argued against? can be tinkered w/ later |>
            select(-hashedEmail, -name) |>
            mutate(gender = as.factor(gender)) |>
            mutate(experience = as.numeric(factor(experience, 
                                              levels = c("Beginner", "Amateur", "Pro", "Veteran")))) |>
            mutate(subscribe = as.numeric(subscribe))

players <- drop_na(players)

players

#visualizing some things that we can see abt. gender makeup of surveyed participants
gender <- players |> ggplot(aes(x = gender, fill = gender)) +
  geom_bar() +
  labs(title = "Gender Distribution of Players", x = "Gender", y = "Count") +
    theme(text = element_text(size = 16))  +
    scale_fill_brewer(palette = 'Pastel1')


experience_gender <- players |> ggplot(aes(x = experience, fill = gender)) +
  geom_bar(position = "fill") +
  labs(color = "Gender", y = "Count", x = "Skill Level") + 
    ggtitle("Self-Reported Experience Levels
            and Genders of Players") +
    theme(text = element_text(size = 16)) +
    scale_fill_brewer(palette = 'Pastel1')

gender
experience_gender




In [ ]:

set.seed(1000) 

#splitting data into testing and training, using the proportion reccomended by TA as feedback to proposal 
gender_split <- initial_split(players, prop = 0.80, strata = gender)
gender_train <- training(gender_split)   
gender_test <- testing(gender_split)


vfold <- vfold_cv(gender_train, v = 2, strata = gender)
k_vals <- tibble(neighbors = seq(from = 1, to = 10, by = 1))

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
                set_engine("kknn") |>
                set_mode("classification")


#excluding playtime bc of outlying data, also debatable
gender_recipe <- recipe(gender ~ experience + subscribe + Age, data = gender_train) |>
                step_scale(all_predictors()) |>
                step_center(all_predictors())


knn_results <- workflow() |>
      add_recipe(gender_recipe) |>
      add_model(knn_tune) |>
      tune_grid(resamples = vfold, grid = k_vals) |>
      collect_metrics() 


accuracies <- knn_results |> 
      filter(.metric == "accuracy")

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
      geom_point() +
      geom_line() +
      labs(x = "Neighbors", y = "Accuracy Estimate") +
      scale_x_continuous(breaks = seq(0, 14, by = 1)) + 
      scale_y_continuous(limits = c(0.4, 1.0)) +
        ggtitle("Accuracy of Algorithm v. K-Valie")

accuracy_versus_k

This graph shows a peak at k = 7 in accuracy. We can run the analysis now, setting k = 7.

In [ ]:
gender_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 7) |>
      set_engine("kknn") |>
      set_mode("classification")

gender_fit <- workflow() |>
      add_recipe(gender_recipe) |>
      add_model(gender_spec) |>
      fit(data = gender_train)

gender_test_predictions <- predict(gender_fit , gender_test) |>
      bind_cols(gender_test)


gender_test_predictions

gender_prediction_accuracy <- gender_test_predictions |>
        metrics(truth = gender, estimate = .pred_class) |>
        filter(.metric == "accuracy") 

gender_prediction_accuracy